<a href="https://colab.research.google.com/github/aduchini/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-14 07:03:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.83MB/s    in 0.2s    

2022-02-14 07:03:59 (4.83 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FU16928EP5TC|          5|            0|          0|   N|                Y|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|                Y|
| R52R85WC6TIAH|          4|           17|         18|   N|                Y|
| R7HOOYTVIB0DS|          5|            0|          0|   N|                Y|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|                Y|
|R1HCST57W334KN|          5|            5|          6|   N|                Y|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|                Y|
|  RH4SXPL4L9QU|          3|            0|          0|   N|                Y|
|R37INWIQA5YW8N|          2|            0|          0|   N|                Y|
|R18GSVAAS3N8GX|          3|            0|          0|   N|     

In [ ]:
# Filtering rows with total_votes greater than or equal to 20
filtered_df = vine_df.filter(vine_df.total_votes >= 20)
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1CY3APKBOVUYU|          1|            9|         33|   N|                Y|
|R2LTI28AN2281A|          1|            0|         22|   N|                Y|
|R3PENXZQNYN29L|          1|            1|         26|   N|                Y|
| R8IGXNWIBN7ZP|          1|            6|         20|   N|                Y|
|R2P8B83Q61L4HS|          1|            0|         22|   N|                Y|
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R257YW4XQLDH1M|          1|            6|         20|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R1TIDPK91S2IRQ|          1|            0|         25|   N|     

In [ ]:
# Filter where the helpful votes proportion is greater than 50%
helpful_votes_df = filtered_df.filter(filtered_df.helpful_votes/filtered_df.total_votes >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|     

In [ ]:
# Filter for reviews that have a written portion only
written_reviews_df = helpful_votes_df.filter(helpful_votes_df.vine == "Y")
written_reviews_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [ ]:
# Filter for reviews that have a written portion only
no_written_reviews_df = helpful_votes_df.filter(helpful_votes_df.vine == "N")
no_written_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|     

In [28]:
count_reviews = helpful_votes_df.count()
count_written = written_reviews_df.count()
count_no_written = no_written_reviews_df.count()
count_no_written_5_star = no_written_reviews_df.filter(no_written_reviews_df.star_rating == 5).count()
count_written_5_star = written_reviews_df.filter(written_reviews_df.star_rating == 5).count()
percentage_no_written = 100 * count_no_written_5_star / (count_no_written_5_star + count_written_5_star)
percentage_5_star = 100 * (count_no_written_5_star + count_written_5_star) / count_reviews
print(f"Total reviews: {count_reviews}")
print(f"Total reviews from non-Vine members: {count_no_written}")
print(f"Total reviews from Vine members: {count_written}")
print(f"Total 5-star from non-Vine members: {count_no_written_5_star}")
print(f"Total 5-star reviews from Vine members: {count_written_5_star}")
print(f"Percentage of 5-star reviews from non-Vine members: {percentage_no_written}%")
print(f"Percentage of 5-star reviews overall: {percentage_5_star}%")


Total reviews: 11371
Total reviews from non-Vine members: 11371
Total reviews from Vine members: 0
Total 5-star from non-Vine members: 4675
Total 5-star reviews from Vine members: 0
Percentage of 5-star reviews from non-Vine members: 100.0%
Percentage of 5-star reviews overall: 41.11335854366371%
